In [1]:
!pip install keras

import importlib

try:
    importlib.import_module('xgboost')
    print("XGBoost ya está instalado en el sistema.")
except ImportError:
    subprocess.run(['pip', 'install', 'xgboost'])
    
try:
    importlib.import_module('lightgbm')
    print("LightGBM ya está instalado en el sistema.")
except ImportError:
    subprocess.run(['pip', 'install', 'lightgbm'])
    
try:
    importlib.import_module('tensorflow')
    print("TensorFlow ya está instalado en el sistema.")
except ImportError:
    subprocess.run(['pip', 'install', 'tensorflow', 'scikit-learn'])

C:\Users\34634\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\34634\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


XGBoost ya está instalado en el sistema.
LightGBM ya está instalado en el sistema.

TensorFlow ya está instalado en el sistema.


In [2]:
# Importaciones necesarias para el correcto funcionamiento de todos lo modelos y demás operaciones con los datos
import joblib
import subprocess
import warnings
import numpy as np
import pandas as pd
import sklearn
import lightgbm as lgb

#Froms
from keras.layers import Dense, LSTM
from keras.models import Sequential
from packaging import version
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, r2_score
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

In [4]:
subprocess.run(['pip', 'install', '--upgrade', 'lightgbm', 'pandas', 'dask'])

CompletedProcess(args=['pip', 'install', '--upgrade', 'lightgbm', 'pandas', 'dask'], returncode=1)

In [18]:
result_df = pd.read_csv("../Fitabase Data 4.12.16-5.12.16/test_train_data.csv")
result_df

,Id,Time,HeartRate,Intensity,Calories
0,2022484408,2016-04-12 07:21:00,101.600000,1,3.32064
1,2022484408,2016-04-12 07:22:00,87.888889,1,3.94326
2,2022484408,2016-04-12 07:23:00,58.000000,0,1.34901
3,2022484408,2016-04-12 07:24:00,58.000000,0,1.03770
4,2022484408,2016-04-12 07:25:00,56.777778,0,1.03770
...,...,...,...,...,...
333141,8877689391,2016-05-12 13:55:00,60.666667,0,1.33353
333142,8877689391,2016-05-12 13:56:00,61.875000,0,1.33353
333143,8877689391,2016-05-12 13:57:00,58.142857,0,1.33353
333144,8877689391,2016-05-12 13:58:00,61.200000,0,1.33353


In [19]:
required_version = '14.0.1'

try:
    pyarrow_version = importlib.import_module('pyarrow').__version__
    if version.parse(pyarrow_version) < version.parse(required_version):
        print(f"PyArrow versión {pyarrow_version} encontrada. Actualizando a la versión {required_version}.")
        subprocess.run(['pip', 'install', f'pyarrow>={required_version}'])
    else:
        print(f"PyArrow versión {pyarrow_version} ya está instalada.")
except ImportError:
    subprocess.run(['pip', 'install', f'pyarrow>={required_version}'])


PyArrow versión 15.0.0 ya está instalada.


In [20]:
result_df['Time'] = pd.to_datetime(result_df['Time'])

result_df['Hour'] = result_df['Time'].dt.hour
result_df['Minutes'] = result_df['Time'].dt.minute
result_df['Weekday'] = result_df['Time'].dt.weekday

features = ['Id', 'Hour', 'Minutes', 'HeartRate']
target = 'Calories'

result_df = result_df.drop('Intensity', axis=1)

X = result_df[features]
y = result_df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.17, random_state=777)

categorical_features = ['Id']
numeric_features = ['Hour', 'Minutes', 'Calories']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ], remainder='passthrough')

# XGBoost: Calories prediction

In [21]:
num_splits = 5 
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_scores = cross_val_score(xgb_model, X, y, scoring='r2', cv=KFold(n_splits=num_splits, shuffle=True, random_state=42))

print(f"\033[94;1mPromedio R2 Score (XGBoost): {xgb_scores.mean()}\033[0m")

Promedio R2 Score (XGBoost): 0.7888487394332563


# LightGBM: Calories prediction

In [23]:
best_params = {'learning_rate': 0.2, 'n_estimators': 200, 'num_leaves': 40}
best_model = lgb.LGBMRegressor(objective='regression', random_state=42, force_row_wise=True, **best_params)

num_splits = 5 
lgb_scores = cross_val_score(best_model, X, y, scoring='r2', cv=KFold(n_splits=num_splits, shuffle=True, random_state=42))

print(f"\033[94;1mPromedio R2 Score (LightGBM): {lgb_scores.mean()}\033[0m")

[LightGBM] [Info] Total Bins 354
[LightGBM] [Info] Number of data points in the train set: 266516, number of used features: 4
[LightGBM] [Info] Start training from score 2.039297
[LightGBM] [Info] Total Bins 354
[LightGBM] [Info] Number of data points in the train set: 266517, number of used features: 4
[LightGBM] [Info] Start training from score 2.038624
[LightGBM] [Info] Total Bins 354
[LightGBM] [Info] Number of data points in the train set: 266517, number of used features: 4
[LightGBM] [Info] Start training from score 2.038586
[LightGBM] [Info] Total Bins 354
[LightGBM] [Info] Number of data points in the train set: 266517, number of used features: 4
[LightGBM] [Info] Start training from score 2.041327
[LightGBM] [Info] Total Bins 354
[LightGBM] [Info] Number of data points in the train set: 266517, number of used features: 4
[LightGBM] [Info] Start training from score 2.040324
Promedio R2 Score (LightGBM): 0.7919600931850364
